### Get current location from IP adress

In [1]:
import requests
import json
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.connect(("8.8.8.8", 80))

full_ip_str = str(s.getsockname()[0])
partial_ip = full_ip_str[:-4]

# URL to send the request to
request_url = 'https://geolocation-db.com/jsonp/' + partial_ip
response = requests.get(request_url)
result = response.content.decode()

# Clean the returned string so it just contains the dictionary data for the IP address
result = result.split("(")[1].strip(")")
# Convert this data into a dictionary
result  = json.loads(result)
my_ip_location = result['latitude'], result['longitude']
#print(result)
#print('------------------')
#print(my_ip_location)
#
## from tuple to float via string
#latstr = str(result['latitude'])
#lat = float(latstr)                                                 
#lonstr = str(result['longitude'])
#lon = float(lonstr)#

## Return: times of Twilight' and day duration:

In [2]:
import datetime as dt
from pytz import timezone
from skyfield import almanac
from skyfield.api import N, E, W, S, wgs84, load
from skyfield.units import Angle

# Figure out local midnight.
tday = dt.datetime.now()
zone = timezone('Europe/Amsterdam')
now = zone.localize(dt.datetime.now())
midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
next_midnight = midnight + dt.timedelta(days=1)


ts = load.timescale()
t0 = ts.from_datetime(midnight)
t1 = ts.from_datetime(next_midnight)
eph = load('de421.bsp') # call to Ephemeris
# Ephemeris: a table or datafile giving the calculated positions of a celestial object at regular intervals.

iploc = wgs84.latlon(result['latitude']* N, result['longitude'] * E)
f = almanac.dark_twilight_day(eph, iploc)
times, events = almanac.find_discrete(t0, t1, f)

print('location:',result['city']+",",result['state']+",", result['country_code'])
print('request time(lt):',tday)
print('timezone:',zone)
lon_ip = '{:.3f}'.format(result["longitude"])
lat_ip = '{:.3f}'.format(result["latitude"])

print("Latitude:   "+str(lat_ip).zfill(6))
print("Longitude: "+str(lon_ip).zfill(7))
#print()
print('________________   ________________________')
previous_e = f(t0).item()
for t, e in zip(times, events):
    tstr = str(t.astimezone(zone))[:16]
    if previous_e < e:
        print(tstr, ' ', almanac.TWILIGHTS[e], 'starts')
    else:
        print(tstr, ' ', almanac.TWILIGHTS[previous_e], 'ends')
    previous_e = e

location: Obdam, North Holland, NL
request time(lt): 2022-05-17 17:19:04.442657
timezone: Europe/Amsterdam
Latitude:   52.675
Longitude: 004.904
________________   ________________________
2022-05-17 03:55   Nautical twilight starts
2022-05-17 04:57   Civil twilight starts
2022-05-17 05:40   Day starts
2022-05-17 21:33   Day ends
2022-05-17 22:17   Civil twilight ends
2022-05-17 23:20   Nautical twilight ends
